In [1]:
# sys path
import sys
sys.path.append('../../')

In [2]:
from holisticai.datasets.synthetic.recruitment import generate_rankings
from holisticai.bias.mitigation.postprocessing.fair_topk.transformer import FairTopK
import pandas as pd

c:\Users\Cristian\anaconda3\envs\metrics\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Top-K Ranking problem

### Generating M synthetic ranking

In [3]:
M = 1
top_n = 20
p = 0.25
ranking = generate_rankings(M, top_n, p)

### Analysing one ranking

In [4]:
ranking['protected']=False
ranking['protected'].iloc[-6:]=True
ranking

C:\Users\Cristian\AppData\Local\Temp\ipykernel_19736\3475268144.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranking['protected'].iloc[-6:]=True


,X,Y,score,protected
0,0,20,20,False
1,0,19,19,False
2,0,18,18,False
3,0,17,17,False
4,0,16,16,False
5,0,15,15,False
6,0,14,14,False
7,0,13,13,False
8,0,12,12,False
9,0,11,11,False


Fair Search Bias Mitigation

In [5]:
top_n = 20
p = 0.5
alpha = 0.1
fs = FairTopK(top_n=top_n, 
              p=p, 
              alpha=alpha, 
              query_col='query_id', 
              doc_col='doc_id', 
              score_col='score', 
              group_col='protected')

Testing if is fair before and after re-rank

In [6]:
print(fs.is_fair(ranking))
re_ranked = fs.transform_ranking(ranking)
print(fs.is_fair(re_ranked))

False
True


In [7]:
result = pd.concat([ranking,re_ranked], axis=1)
result

,X,Y,score,protected,X,Y,score,protected
0,0,20,20,False,0,20,20,False
1,0,19,19,False,0,19,19,False
2,0,18,18,False,0,18,18,False
3,0,17,17,False,0,17,17,False
4,0,16,16,False,0,6,6,True
5,0,15,15,False,0,16,16,False
6,0,14,14,False,0,15,15,False
7,0,13,13,False,0,5,5,True
8,0,12,12,False,0,14,14,False
9,0,11,11,False,0,13,13,False


You can use FairTopK  and pass ranking and p_attr (protected groups) separated or in the same dataframe.

In [9]:
# Synthetic data
M = 10000
top_n = 20
p = 0.25
rankings , p_attr = generate_rankings(M, top_n, p, return_p_attr=True)


# Bias Mitigation Post-processing
top_n = 20
p = 0.5
alpha = 0.1
fs = FairTopK(top_n=top_n, 
              p=p, 
              alpha=alpha, 
              query_col='X', 
              doc_col='Y', 
              score_col='score', 
              group_col='protected')

# Transform passing separated arguments
re_rankings = fs.transform(rankings, p_attr=p_attr)

In [10]:
# Synthetic data
M = 10000
top_n = 20
p = 0.25
rankings = generate_rankings(M, top_n, p, return_p_attr=False)


# Bias Mitigation Post-processing
top_n = 20
p = 0.5
alpha = 0.1
fs = FairTopK(top_n=top_n, 
              p=p, 
              alpha=alpha, 
              query_col='X', 
              doc_col='Y', 
              score_col='score', 
              group_col='protected')

# Transform passing all info in the same dataframe
re_rankings = fs.transform(rankings)

In [12]:
queri_id = 10
old = rankings[rankings['X']==queri_id]
old.columns = pd.MultiIndex.from_tuples([['Old Rank',col] for col in old.columns])
new = re_rankings[re_rankings['X']==queri_id]
new.columns = pd.MultiIndex.from_tuples([['New Rank',col] for col in new.columns])
result = pd.concat([old,new], axis=1)
result

Old Rank                     New Rank                    
           X   Y score protected        X   Y score protected
200       10  20    20     False       10  20    20     False
201       10  19    19     False       10  19    19     False
202       10  18    18     False       10  18    18     False
203       10  17    17     False       10  17    17     False
204       10  16    16     False       10  13    13      True
205       10  15    15     False       10  16    16     False
206       10  14    14     False       10  15    15     False
207       10  13    13      True       10  12    12      True
208       10  12    12      True       10  14    14     False
209       10  11    11     False       10  11    11     False
210       10  10    10     False       10   9     9      True
211       10   9     9      True       10  10    10     False
212       10   8     8     False       10   3     3      True
213       10   7     7     False       10   8     8     False
214       10   6     6     False       10   7     7     False
215       10   5     5     False       10   6     6     False
216       10   4     4     False       10   5     5     False
217       10   3     3      True       10   4     4     False
218       10   2     2     False       10   2     2     False
219       10   1     1     False       10   1     1     False